# BAM/SAM Alignment Feature Extraction

This notebook demonstrates how to:
1. Load BAM files with multithreaded decompression
2. Extract alignment features for machine learning
3. Filter alignments by quality and pairing
4. Count chimeric reads
5. Export to Arrow/Parquet for analysis

## Installation

```bash
pip install deepbiop
```

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import deepbiop as dbp

## 1. Loading BAM Files

DeepBioP provides multithreaded BAM decompression for high performance.

BAM (Binary Alignment/Map) format stores aligned sequencing reads:
- Reference sequence alignment position
- CIGAR string (alignment operations)
- Mapping quality (MAPQ)
- SAM flags (paired, mapped, supplementary, etc.)
- Optional tags (edit distance, alignment score, etc.)

In [ ]:
# Open BAM file with 4 decompression threads
# Adjust thread count based on CPU cores
reader = dbp.BamReader("alignments.bam", threads=4)

print("BAM reader created with multithreaded decompression")
print("Ready to extract alignment features")

## 2. Extracting Alignment Features

AlignmentFeatures captures key quality metrics:

| Feature | Description | ML Use Case |
|---------|-------------|-------------|
| mapping_quality | Phred-scaled confidence (0-255) | Filter low-quality alignments |
| aligned_length | Total aligned bases | Normalize coverage |
| num_matches | Matching bases | Calculate identity |
| num_insertions | Insertion events | Detect indels |
| num_deletions | Deletion events | Variant calling features |
| num_soft_clips | Soft-clipped bases | Structural variant signals |
| edit_distance | Total mismatches + indels | Alignment quality metric |

In [ ]:
# Extract features from all alignments
features = reader.extract_features()

print(f"Extracted features from {len(features)} alignments")
print("\nFirst 5 alignments:")
for i, feat in enumerate(features[:5], 1):
    print(f"\nAlignment {i}:")
    print(f"  Mapping quality: {feat.mapping_quality}")
    print(f"  Aligned length: {feat.aligned_length}bp")
    print(f"  Matches: {feat.num_matches}")
    print(f"  Identity: {feat.identity():.3f}")
    print(f"  Is mapped: {feat.is_mapped}")
    print(f"  Is paired: {feat.is_paired}")

## 3. Computing Feature Statistics

### Identity
Identity = matches / (matches + mismatches + insertions + deletions)

High identity (>0.95) indicates good alignment quality.

### Indel Rate
Indel rate = (insertions + deletions) / aligned_length

High indel rates may indicate sequencing errors or structural variants.

In [ ]:
# Calculate statistics across all alignments
identities = [feat.identity() for feat in features if feat.is_mapped]
mapq_scores = [feat.mapping_quality for feat in features if feat.is_mapped]
aligned_lengths = [feat.aligned_length for feat in features if feat.is_mapped]

print("Alignment Statistics:")
print(f"  Mean identity: {np.mean(identities):.3f}")
print(f"  Median identity: {np.median(identities):.3f}")
print(f"  Mean MAPQ: {np.mean(mapq_scores):.1f}")
print(f"  Mean aligned length: {np.mean(aligned_lengths):.1f}bp")
print(f"  Mapped reads: {sum(1 for f in features if f.is_mapped)} / {len(features)}")

## 4. Visualizing Alignment Quality

Quality distributions help identify:
- Poor alignment regions (low MAPQ)
- Mapping biases
- Outlier samples

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Mapping quality distribution
axes[0, 0].hist(mapq_scores, bins=50, color="skyblue", edgecolor="black")
axes[0, 0].set_xlabel("Mapping Quality (MAPQ)")
axes[0, 0].set_ylabel("Count")
axes[0, 0].set_title("Mapping Quality Distribution")
axes[0, 0].axvline(x=30, color="r", linestyle="--", label="Q30 threshold")
axes[0, 0].legend()

# Identity distribution
axes[0, 1].hist(identities, bins=50, color="lightgreen", edgecolor="black")
axes[0, 1].set_xlabel("Alignment Identity")
axes[0, 1].set_ylabel("Count")
axes[0, 1].set_title("Alignment Identity Distribution")
axes[0, 1].axvline(x=0.95, color="r", linestyle="--", label="95% threshold")
axes[0, 1].legend()

# Aligned length distribution
axes[1, 0].hist(aligned_lengths, bins=50, color="salmon", edgecolor="black")
axes[1, 0].set_xlabel("Aligned Length (bp)")
axes[1, 0].set_ylabel("Count")
axes[1, 0].set_title("Aligned Length Distribution")

# MAPQ vs Identity scatter
axes[1, 1].scatter(mapq_scores, identities, alpha=0.3, s=10)
axes[1, 1].set_xlabel("Mapping Quality (MAPQ)")
axes[1, 1].set_ylabel("Alignment Identity")
axes[1, 1].set_title("MAPQ vs Identity")
axes[1, 1].axhline(y=0.95, color="r", linestyle="--", alpha=0.5)
axes[1, 1].axvline(x=30, color="r", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

## 5. Filtering High-Quality Alignments

Common quality thresholds:
- **MAPQ ≥ 30**: High-confidence unique mapping (error rate 0.001)
- **MAPQ ≥ 20**: Good quality (error rate 0.01)
- **MAPQ ≥ 10**: Moderate quality (error rate 0.1)
- **MAPQ = 0**: Multi-mapping or unmapped

For ML pipelines, typically use MAPQ ≥ 30 for training data.

In [ ]:
# Filter by mapping quality (MAPQ >= 30)
reader2 = dbp.BamReader("alignments.bam", threads=4)
high_qual_count = reader2.filter_by_mapping_quality(30)

print(f"High-quality alignments (MAPQ ≥ 30): {high_qual_count}")
print(f"Percentage of total: {high_qual_count / len(features) * 100:.1f}%")

# Compare quality tiers
thresholds = [0, 10, 20, 30, 40]
counts = []

for threshold in thresholds:
    count = sum(1 for f in features if f.mapping_quality >= threshold)
    counts.append(count)
    print(f"MAPQ ≥ {threshold:2d}: {count:6d} ({count / len(features) * 100:5.1f}%)")

## 6. Proper Pair Analysis

For paired-end sequencing, proper pairs have:
- Both reads mapped
- Same reference sequence
- Expected orientation (forward-reverse)
- Insert size within expected range

Improper pairs may indicate:
- Structural variants
- Chimeric reads
- Sequencing artifacts

In [ ]:
# Analyze proper pairs with 1000bp max insert size
max_insert_size = 1000

proper_pairs = [f for f in features if f.is_proper_pair(max_insert_size)]
paired_reads = [f for f in features if f.is_paired]

print(f"Paired reads: {len(paired_reads)}")
print(f"Proper pairs: {len(proper_pairs)}")
print(f"Proper pair rate: {len(proper_pairs) / len(paired_reads) * 100:.1f}%")

# Insert size distribution for proper pairs
insert_sizes = [abs(f.template_length) for f in proper_pairs if f.template_length != 0]

if insert_sizes:
    plt.figure(figsize=(10, 5))
    plt.hist(insert_sizes, bins=50, color="mediumpurple", edgecolor="black")
    plt.xlabel("Insert Size (bp)")
    plt.ylabel("Count")
    plt.title("Insert Size Distribution (Proper Pairs)")
    plt.axvline(
        x=np.mean(insert_sizes),
        color="r",
        linestyle="--",
        label=f"Mean: {np.mean(insert_sizes):.0f}bp",
    )
    plt.legend()
    plt.show()

    print("\nInsert size statistics:")
    print(f"  Mean: {np.mean(insert_sizes):.1f}bp")
    print(f"  Median: {np.median(insert_sizes):.1f}bp")
    print(f"  Std dev: {np.std(insert_sizes):.1f}bp")

## 7. Chimeric Read Detection

Chimeric reads span multiple loci:
- Primary alignment + supplementary alignment(s)
- Indicate gene fusions, structural variants, or artifacts

SAM flags for chimeric detection:
- `0x800`: Supplementary alignment
- Primary + supplementary from same read = chimeric

In [ ]:
# Count chimeric reads
reader3 = dbp.BamReader("alignments.bam", threads=4)
chimeric_count = reader3.count_chimeric_reads()

print(f"Chimeric reads detected: {chimeric_count}")
print(f"Chimeric rate: {chimeric_count / len(features) * 100:.2f}%")

# Typical chimeric rates:
# - Normal samples: <0.1%
# - Cancer samples with fusions: 0.1-1%
# - High chimeric rate may indicate library prep issues

## 8. CIGAR String Analysis

CIGAR operations describe alignment:
- **M**: Match or mismatch
- **I**: Insertion (in read)
- **D**: Deletion (in read)
- **S**: Soft clip (not aligned)
- **H**: Hard clip (not present)
- **N**: Skipped region (splice junction)

High soft clip rates may indicate:
- Adapter contamination
- Structural variants
- Low-quality ends

In [ ]:
# Analyze CIGAR operations
total_insertions = sum(f.num_insertions for f in features if f.is_mapped)
total_deletions = sum(f.num_deletions for f in features if f.is_mapped)
total_soft_clips = sum(f.num_soft_clips for f in features if f.is_mapped)
total_hard_clips = sum(f.num_hard_clips for f in features if f.is_mapped)
total_aligned = sum(f.aligned_length for f in features if f.is_mapped)

print("CIGAR Operation Statistics:")
print(f"  Total aligned bases: {total_aligned}")
print(
    f"  Insertions: {total_insertions} ({total_insertions / total_aligned * 100:.3f}%)"
)
print(f"  Deletions: {total_deletions} ({total_deletions / total_aligned * 100:.3f}%)")
print(
    f"  Soft clips: {total_soft_clips} ({total_soft_clips / total_aligned * 100:.3f}%)"
)
print(f"  Hard clips: {total_hard_clips}")

# Visualize operation frequencies
operations = ["Insertions", "Deletions", "Soft Clips"]
counts = [total_insertions, total_deletions, total_soft_clips]

plt.figure(figsize=(8, 6))
plt.bar(operations, counts, color=["indianred", "steelblue", "orange"])
plt.ylabel("Count")
plt.title("CIGAR Operation Distribution")
plt.yscale("log")
plt.show()

## 9. Export to Arrow/Parquet

Export alignment features to columnar format for:
- pandas/polars analysis
- DuckDB queries
- ML feature engineering

Arrow format provides:
- Zero-copy sharing
- Language interoperability
- Efficient columnar access

In [ ]:
# Export to Arrow table
import pyarrow as pa
import pyarrow.parquet as pq

# Convert features to dictionary format
data = {
    "mapping_quality": [f.mapping_quality for f in features],
    "aligned_length": [f.aligned_length for f in features],
    "num_matches": [f.num_matches for f in features],
    "num_insertions": [f.num_insertions for f in features],
    "num_deletions": [f.num_deletions for f in features],
    "num_soft_clips": [f.num_soft_clips for f in features],
    "identity": [f.identity() for f in features],
    "is_mapped": [f.is_mapped for f in features],
    "is_paired": [f.is_paired for f in features],
}

# Create Arrow table
table = pa.table(data)

print(f"Arrow table created: {table.shape[0]} rows, {table.shape[1]} columns")
print("\nSchema:")
print(table.schema)

# Write to Parquet
pq.write_table(table, "alignment_features.parquet")
print("\nExported to alignment_features.parquet")

## 10. Integration with pandas

Convert to pandas for exploratory analysis and ML feature engineering.

In [ ]:
# Convert Arrow to pandas
df = table.to_pandas()

print("DataFrame preview:")
print(df.head())

print("\nDescriptive statistics:")
print(df.describe())

# Filter high-quality alignments
high_quality = df[(df["mapping_quality"] >= 30) & (df["identity"] >= 0.95)]
print(f"\nHigh-quality alignments: {len(high_quality)} / {len(df)}")

# Feature engineering: indel rate
df["indel_rate"] = (df["num_insertions"] + df["num_deletions"]) / df["aligned_length"]
df["clip_rate"] = df["num_soft_clips"] / df["aligned_length"]

print("\nEngineered features:")
print(df[["indel_rate", "clip_rate"]].describe())

## 11. Machine Learning Feature Matrix

Prepare feature matrix for ML models:
- Normalize features
- Handle missing values
- Create binary quality labels

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select numeric features
feature_columns = [
    "mapping_quality",
    "aligned_length",
    "num_matches",
    "num_insertions",
    "num_deletions",
    "num_soft_clips",
    "identity",
    "indel_rate",
    "clip_rate",
]

X = df[feature_columns].values

# Create binary label (high quality = MAPQ >= 30 and identity >= 0.95)
y = ((df["mapping_quality"] >= 30) & (df["identity"] >= 0.95)).astype(int)

print(f"Feature matrix shape: {X.shape}")
print("Label distribution:")
print(f"  High quality: {y.sum()} ({y.sum() / len(y) * 100:.1f}%)")
print(
    f"  Low quality: {(~y.astype(bool)).sum()} ({(~y.astype(bool)).sum() / len(y) * 100:.1f}%)"
)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\nScaled feature matrix ready for ML models")
print(f"Shape: {X_scaled.shape}")
print(f"Mean: {X_scaled.mean(axis=0)}")
print(f"Std: {X_scaled.std(axis=0)}")

## Summary

DeepBioP provides comprehensive BAM processing for ML pipelines:

| Feature | Purpose | ML Application |
|---------|---------|----------------|
| Multithreaded reading | Fast I/O | Large-scale data processing |
| AlignmentFeatures | Quality metrics | Feature engineering |
| Filtering | Quality control | Training data curation |
| Chimeric detection | Structural variants | Fusion gene detection |
| Arrow export | Interoperability | Integration with pandas/DuckDB |
| Batch processing | Efficiency | Production pipelines |

Benefits:
- ✅ Fast multithreaded BAM decompression
- ✅ Rich alignment quality metrics
- ✅ Flexible filtering and analysis
- ✅ Zero-copy Arrow export
- ✅ Ready for scikit-learn/PyTorch